In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [6]:
# Cargar los datos de la serie de tiempo
data = pd.read_csv('datos\we_eurusd.csv') 
values = data['4. close'].values
values = values.reshape(-1, 1)

In [7]:
# Escalar los datos al rango (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [8]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(scaled) * 0.80)
train, test = scaled[0:train_size, :], scaled[train_size:len(scaled), :]

In [9]:
# Crear los conjuntos de datos para entrenamiento y prueba
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [10]:
look_back = 7  # Número de pasos anteriores para considerar
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [11]:
# Reshape de entrada [muestras, pasos de tiempo, características]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [12]:
# Crear y compilar el modelo LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [13]:
# Entrenar el modelo
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
792/792 - 3s - loss: 0.0053 - 3s/epoch - 4ms/step
Epoch 2/100
792/792 - 2s - loss: 0.0023 - 2s/epoch - 2ms/step
Epoch 3/100
792/792 - 2s - loss: 0.0019 - 2s/epoch - 2ms/step
Epoch 4/100
792/792 - 2s - loss: 0.0015 - 2s/epoch - 2ms/step
Epoch 5/100
792/792 - 2s - loss: 0.0012 - 2s/epoch - 3ms/step
Epoch 6/100
792/792 - 2s - loss: 0.0011 - 2s/epoch - 2ms/step
Epoch 7/100
792/792 - 2s - loss: 9.8831e-04 - 2s/epoch - 2ms/step
Epoch 8/100
792/792 - 2s - loss: 9.6706e-04 - 2s/epoch - 2ms/step
Epoch 9/100
792/792 - 2s - loss: 9.7731e-04 - 2s/epoch - 3ms/step
Epoch 10/100
792/792 - 2s - loss: 8.9622e-04 - 2s/epoch - 2ms/step
Epoch 11/100
792/792 - 2s - loss: 9.0648e-04 - 2s/epoch - 3ms/step
Epoch 12/100
792/792 - 3s - loss: 9.6545e-04 - 3s/epoch - 3ms/step
Epoch 13/100
792/792 - 2s - loss: 8.9379e-04 - 2s/epoch - 3ms/step
Epoch 14/100
792/792 - 2s - loss: 8.7282e-04 - 2s/epoch - 2ms/step
Epoch 15/100
792/792 - 2s - loss: 8.7114e-04 - 2s/epoch - 2ms/step
Epoch 16/100
792/792 - 2s - 

In [14]:
# Realizar pronósticos en el conjunto de prueba
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

7/7 [==============================] - 0s 2ms/step


In [15]:
# Calcular métricas de evaluación
rmse = np.sqrt(mean_squared_error(testY[0], testPredict[:, 0]))
mae = mean_absolute_error(testY[0], testPredict[:, 0])
mape = np.mean(np.abs((testY[0] - testPredict[:, 0]) / testY[0])) * 100
theil = np.sqrt((rmse**2) / (np.mean(testY[0]**2)))
theil_u2 = np.sqrt(((rmse**2) + (mae**2)) / (np.mean(testY[0]**2)))
symmetric_mape = (100 / len(testY[0])) * np.sum(2 * np.abs(testPredict[:, 0] - testY[0]) / (np.abs(testPredict[:, 0]) + np.abs(testY[0])))

print("RMSE:", rmse)
print("MAE:", mae)
print("MAPE:", mape)
print("Theil:", theil)
print("Theil U2:", theil_u2)
print("Symmetric MAPE:", symmetric_mape)


RMSE: 0.014797798633034443
MAE: 0.011360691702452352
MAPE: 1.0288090219516333
Theil: 0.013206760494356125
Theil U2: 0.016649985851230877
Symmetric MAPE: 1.0289352809208951
